<a href="https://colab.research.google.com/github/llowoll/2019_cau_oss_hackathon/blob/master/hackathon_team04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
 *  단, 사용할 데이터셋에 따라 is_mnist만 수정
*   모든 구현은 [2. 데이터 전처리]와 [3. 모델 생성]에서만 진행
 *  데이터 전처리 후 트레이닝, 데이터 셋은 x_train_after, x_test_after 변수명을 유지해주세요.
 *  데이터셋이 달라져도 같은 모델 구조를 유지하여야함.
*   [4. 모델 저장]과 [5. 모델 로드 및 평가]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *  트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
 *  team_name을 제외한 다른 부분은 수정하지 말 것
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  모델 구조 파일 (model_structure_teamXX.json)
 *  모델 weight 파일 (model_weight_teamXX.h5)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
* 제출 기한: **오후 6시**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2019_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 테스트 데이터 셋 분류 정확도
 *  model.evaluate(x_test, y_test)
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  두 개의 데이터셋에 대해 다른 모델 구조를 사용한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예. 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성 
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils

import numpy as np

is_mnist = False;

# 데이터셋 로드
# x_train, y_train: 트레이닝 데이터 및 레이블
# x_test, y_test: 테스트 데이터 및 레이블
if is_mnist:
  data_type = 'mnist'
  (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data() # fashion MNIST 데이터셋인 경우,
else:
  data_type = 'cifar10'
  (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data() # cifar10 데이터셋인 경우,


# 분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# 총 클래스 개수
num_classes = y_test.shape[1]

# 인풋 데이터 타입
input_shape = x_test.shape[1:]

# **2. 데이터 전처리**


In [0]:
# 데이터 전처리 (예: normalization)
x_train_after = x_train / 255.0
x_test_after = x_test / 255.0

if is_mnist:
  x_train_after = x_train_after.reshape(x_train_after.shape[0], 28, 28, 1)
  x_test_after = x_test_after.reshape(x_test_after.shape[0], 28, 28, 1)
  input_shape = x_test_after.shape[1:]

# **3. 모델 생성**



In [0]:
# 순차 모델 생성
model = keras.Sequential()

# 1st hidden layer: convolutional-layer
# (# of inputs = 28x28x1, # of outputs = 30x28x28x1, padding = same, activation function = leaky_relu)
model.add(keras.layers.Conv2D(30, (3, 3), padding='same', activation=tf.nn.leaky_relu, input_shape=input_shape))

# Normalize Batch
model.add(keras.layers.BatchNormalization())

# 2nd hidden layer: convolutional-layer
# (#of inputs = 30x28x28x1, # of outputs = 30x28x28x1, padding = same, activation function = leak_relu)
model.add(keras.layers.Conv2D(30, (3, 3), padding='same', activation=tf.nn.leaky_relu))

# Normalize Batch
model.add(keras.layers.BatchNormalization())

# 3rd hidden layer: maxpool-layer
# (#of inputs = 30x28x28x1, # of outputs = 30x14x14x1)
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

# 4th hidden layer: convolutional-layer
# (#of inputs = 30x14x14x1, # of outputs = 35x14x14x1, padding = same, activation function = leak_relu)
model.add(keras.layers.Conv2D(35, (3, 3), padding='same', activation=tf.nn.leaky_relu))

# Normalize Batch
model.add(keras.layers.BatchNormalization())

# 5th hidden layer: convolutional-layer
# (#of inputs = 35x14x14x1, # of outputs = 35x14x14x1, padding = same, activation function = leak_relu)
model.add(keras.layers.Conv2D(35, (3, 3), padding='same', activation=tf.nn.leaky_relu))

# Normalize Batch
model.add(keras.layers.BatchNormalization())

# 6th hidden layer: maxpool-layer
# (#of inputs = 35x14x14x1, # of outputs = 35x7x7x1)
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

# 7th hidden layer: convolutional-layer
# (#of inputs = 35x7x7x1, # of outputs = 40x7x7x1, padding = same, activation function = leak_relu)
model.add(keras.layers.Conv2D(40, (3, 3), padding='same', activation=tf.nn.leaky_relu))

# Normalize Batch
model.add(keras.layers.BatchNormalization())

# 8th hidden layer: convolutional-layer
# (#of inputs = 40x7x7x1, # of outputs = 40x7x7x1, padding = same, activation function = leak_relu)
model.add(keras.layers.Conv2D(40, (3, 3), padding='same', activation=tf.nn.leaky_relu))

# Normalize Batch
model.add(keras.layers.BatchNormalization())

# 9th hidden layer: maxpool-layer
# (#of inputs = 40x7x7x1, #of outputs = 40x4x4x1)
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

# Flatten layer: 40x4x4x1 to 640 1d vector layer
model.add(keras.layers.Flatten())

# 10th hidden layer: fully-connected layer
# (#of inputs = 640, #of outputs = 128, activation function = leak_relu)
model.add(keras.layers.Dense(128, activation=tf.nn.leaky_relu))

# Normalize Batch
model.add(keras.layers.BatchNormalization())

# 11th hidden layer: fully-connected layer
# (# of inputs = 128, # of outputs = 32, activation function = leak_relu)
model.add(keras.layers.Dense(32, activation=tf.nn.leaky_relu))

# Normalize Batch
model.add(keras.layers.BatchNormalization())

# Output layer: fully-connected layer
# (# of inputs = 32, # of outputs = 10, activation funtction = leak_relu)
model.add(keras.layers.Dense(num_classes, activation='softmax'))

# Compile Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Flip Image Horizontally
train_datagen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True)

# Train model
model.fit_generator(
    train_datagen.flow(x_train_after, y_train, batch_size = 16),
    steps_per_epoch = 1000,
    epochs = 20,
    validation_data = (x_test_after, y_test),
    validation_steps = 10)

Epoch 1/20
1000/1000 [==============================] - 18s 18ms/step - loss: 1.5596 - acc: 0.4391 - val_loss: 1.6484 - val_acc: 0.4527
Epoch 2/20
1000/1000 [==============================] - 13s 13ms/step - loss: 1.2428 - acc: 0.5594 - val_loss: 1.1741 - val_acc: 0.5830
Epoch 3/20
1000/1000 [==============================] - 13s 13ms/step - loss: 1.0892 - acc: 0.6147 - val_loss: 1.2617 - val_acc: 0.5632
Epoch 4/20
1000/1000 [==============================] - 13s 13ms/step - loss: 0.9801 - acc: 0.6598 - val_loss: 0.9016 - val_acc: 0.6873
Epoch 5/20
1000/1000 [==============================] - 13s 13ms/step - loss: 0.9351 - acc: 0.6758 - val_loss: 0.9792 - val_acc: 0.6567
Epoch 6/20
1000/1000 [==============================] - 13s 13ms/step - loss: 0.9098 - acc: 0.6824 - val_loss: 0.8657 - val_acc: 0.7059
Epoch 7/20
1000/1000 [==============================] - 13s 13ms/step - loss: 0.8520 - acc: 0.7021 - val_loss: 0.9220 - val_acc: 0.6759
Epoch 8/20
1000/1000 [==========================

# **4. 모델 저장**

In [0]:
save_path = '/content/'
team_name = 'team04'

# 모델의 weight 값만 저장합니다.
model.save_weights(save_path + 'model_weight_' + data_type + '_' + team_name + '.h5')

# 모델의 구조만을 저장합니다.
model_json = model.to_json()
with open(save_path + 'model_structure_' + data_type + '_' + team_name + '.json', 'w') as json_file : 
    json_file.write(model_json)

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_' + data_type + '_' + team_name + '.h5')

# **5. 모델 로드 및 평가**

In [0]:
save_path = '/content/'
team_name = 'team04'

model = keras.models.load_model(save_path + 'model_entire_' + data_type + '_' + team_name + '.h5')
model.evaluate(x_test_after, y_test)